In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

## s*10 (-2.5)

In [3]:
k = 5
s = -1/(k-1)*10
C_num = 3
rep = 5

In [4]:
s

-2.5

In [12]:
r_s2_test = []
r_s5_test = []
r_s10_test = []
r_s10_l2_test = []

In [13]:
f_s2_test = []
f_s5_test = []
f_s10_test = []
f_s10_l2_test = []

### s2

In [30]:
for label in range(rep):
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s2_soap/'
    file_data = rountine + 'all_data_0309(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])

    file_data = rountine + 'y_label_0309(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0309(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n_test = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0309(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y_test),1))
    
    
    
    
    my_omega = np.random.randn(x_dim,k)
    my_b = np.random.randn(k)
    C_num = 0.2
    correct_num = 0
    iter_num = 15  ###总迭代次数

    train_cr = []
    test_cr = []
    test_f1 = [] ######
    
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        if esti.argmax() == my_y[i]:
            correct_num += 1

    for iter in range(iter_num):
        try:
            my_beta = np.zeros((n,k))
            ##### beta
            for i in range(n):
                pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
                pred_copy = pred_iter.copy()
                pred_iter[int(my_y[i])] = -float('inf')
                j = pred_iter.argmax()

                if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
                    my_beta[i,j] = C_num

            ##### function1
            my_alpha = cp.Variable((n,k))
            obj_function = 0

            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

                s1 += cp.sum_squares(s1_1 - s1_2)

            s2 = 0
            for i in range(n):
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s2 += my_alpha[i,j2]

            obj_function = 0.5 * s1 - s2

            ##### constrains1
            constrains = []
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j])

                constrains += [(s1_1 - s1_2) == 0]

            for i in range(n):
                s1 = 0
                for j in range(k):
                    if j != my_y[i]:
                        s1 += my_alpha[i,j]
                        constrains += [s1 >= 0]
                        constrains += [s1 <= C_num]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains += [my_alpha[i,j] >= 0]

            prob = cp.Problem(cp.Minimize(obj_function), constrains)
            prob.solve()

            ##### calculate omega
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                my_omega[:,j] = s1_1 - s1_2

            #print(my_omega)

            #####function2
            my_eta = cp.Variable(n)
            my_b = cp.Variable(k)

            obj_function_2 = 0
            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += my_beta[i,j2]

                    if my_y[i] != j:
                        s1_2 += my_beta[i,j2]

                s1 += (s1_1 - s1_2) * my_b[j]

            obj_function_2 = C_num * cp.sum(my_eta) + s1

            ##### constrains2

            constrains_2 = []
            f_sim = np.dot(my_x, my_omega)

            for i in range(n):
                constrains_2 += [my_eta[i] >= 0]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

            constrains_2 += [cp.sum(my_b) == 0]

            prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)

            prob_2.solve()
            my_b = my_b.value
            #print(my_b)

            ### train set correct-ratio ###

            correct_num = 0
            esti_mat = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x[i,:], my_omega) + my_b
                esti_mat[i] = esti.argmax()
                if esti.argmax() == my_y[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat = np.zeros((k,k))
            for i in range(n):
                cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
            #print(cf_mat)


            train_cr.append(correct_num/n)


            ### test set correct-ratio ###
            correct_num = 0
            esti_mat_test = np.zeros(n_test)
            for i in range(n_test):
                esti = np.dot(my_x_test[i,:], my_omega) + my_b
                esti_mat_test[i] = esti.argmax()  ######
                if esti.argmax() == my_y_test[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat_test = np.zeros((k,k))
            for i in range(n_test):
                cf_mat_test[int(my_y_test[i]),int(esti_mat_test[i])] += 1
            #print(cf_mat)

            test_cr.append(correct_num/n_test)
            test_f1.append(f1_score(my_y_test, esti_mat_test, average='macro')) ######

    
        except TypeError:
            print('can not calculate')
            if(len(test_cr) == 0):
                break
            print(train_cr[-1])
            print(cf_mat)
            print(max(test_cr))
            print(cf_mat_test)
            
            break
        
    if((iter == (iter_num - 1)) and (iter != 0)):
        print(train_cr[-1])
        print(cf_mat)

        print(max(test_cr))
        print(cf_mat_test)
        
        r_s2_test.append(max(test_cr))  
        f_s2_test.append(max(test_f1))  ######

1
0.5666666666666667
[[ 6.  0.  0.  7. 16.]
 [ 1. 46.  2.  6.  7.]
 [ 1. 24. 45.  3.  8.]
 [13.  5.  1. 24.  9.]
 [ 1.  0. 18.  8. 49.]]
0.58
[[ 4.  0.  0.  3.  5.]
 [ 0. 12.  0.  3.  2.]
 [ 0. 14. 10.  1.  3.]
 [ 6.  3.  0. 10.  2.]
 [ 1.  0.  5.  5. 11.]]
2
0.36
[[10. 13.  0.  5. 11.]
 [ 4.  1. 33. 16.  3.]
 [ 6.  5. 46.  0. 29.]
 [ 2.  9. 11. 30.  9.]
 [17. 15.  4.  0. 21.]]
0.69
[[ 4.  4.  0.  5.  2.]
 [ 0.  0.  5.  2.  2.]
 [ 2.  0. 19.  0. 13.]
 [ 2.  5.  2. 10.  4.]
 [ 7.  6.  0.  0.  6.]]
3
0.41333333333333333
[[ 8. 11.  0. 12. 20.]
 [ 9. 13. 25.  3.  0.]
 [ 5.  8. 52.  4.  9.]
 [13. 25.  6.  2. 10.]
 [ 1.  0.  2. 13. 49.]]
0.52
[[ 2.  1.  0.  2.  3.]
 [ 2.  4.  8.  1.  0.]
 [ 2.  1. 25.  1.  1.]
 [ 4. 13.  1.  2.  4.]
 [ 1.  1.  0.  2. 19.]]
4
0.5033333333333333
[[ 7. 13.  0.  6. 15.]
 [16.  1. 14. 21.  1.]
 [ 9.  3. 64.  3.  6.]
 [10.  8.  3. 27.  8.]
 [ 3.  9.  1.  0. 52.]]
0.58
[[ 1.  3.  0.  4.  1.]
 [ 3.  0.  4.  7.  0.]
 [ 3.  0. 25.  0.  5.]
 [ 7.  1.  0. 14.  2.]
 [ 0.

SolverError: Solver 'OSQP' failed. Try another solver, or solve with verbose=True for more information.

In [ ]:
print(r_s2_test)
print(np.mean(r_s2_test))
print(np.std(r_s2_test))

print(f_s2_test)
print(np.mean(f_s2_test))
print(np.std(f_s2_test))

### s5

In [29]:
for label in range(rep):
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s5_soap/'
    file_data = rountine + 'all_data_0309(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])

    file_data = rountine + 'y_label_0309(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0309(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n_test = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0309(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y_test),1))
    
    
    
    
    my_omega = np.random.randn(x_dim,k)
    my_b = np.random.randn(k)
    C_num = 0.2
    correct_num = 0
    iter_num = 15  ###总迭代次数

    train_cr = []
    test_cr = []
    test_f1 = [] ###
    
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        if esti.argmax() == my_y[i]:
            correct_num += 1

    for iter in range(iter_num):
        try:
            my_beta = np.zeros((n,k))
            ##### beta
            for i in range(n):
                pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
                pred_copy = pred_iter.copy()
                pred_iter[int(my_y[i])] = -float('inf')
                j = pred_iter.argmax()

                if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
                    my_beta[i,j] = C_num

            ##### function1
            my_alpha = cp.Variable((n,k))
            obj_function = 0

            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

                s1 += cp.sum_squares(s1_1 - s1_2)

            s2 = 0
            for i in range(n):
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s2 += my_alpha[i,j2]

            obj_function = 0.5 * s1 - s2

            ##### constrains1
            constrains = []
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j])

                constrains += [(s1_1 - s1_2) == 0]

            for i in range(n):
                s1 = 0
                for j in range(k):
                    if j != my_y[i]:
                        s1 += my_alpha[i,j]
                        constrains += [s1 >= 0]
                        constrains += [s1 <= C_num]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains += [my_alpha[i,j] >= 0]

            prob = cp.Problem(cp.Minimize(obj_function), constrains)
            prob.solve()

            ##### calculate omega
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                my_omega[:,j] = s1_1 - s1_2

            #print(my_omega)

            #####function2
            my_eta = cp.Variable(n)
            my_b = cp.Variable(k)

            obj_function_2 = 0
            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += my_beta[i,j2]

                    if my_y[i] != j:
                        s1_2 += my_beta[i,j2]

                s1 += (s1_1 - s1_2) * my_b[j]

            obj_function_2 = C_num * cp.sum(my_eta) + s1

            ##### constrains2

            constrains_2 = []
            f_sim = np.dot(my_x, my_omega)

            for i in range(n):
                constrains_2 += [my_eta[i] >= 0]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

            constrains_2 += [cp.sum(my_b) == 0]

            prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)

            prob_2.solve()
            my_b = my_b.value
            #print(my_b)

            ### train set correct-ratio ###

            correct_num = 0
            esti_mat = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x[i,:], my_omega) + my_b
                esti_mat[i] = esti.argmax()
                if esti.argmax() == my_y[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat = np.zeros((k,k))
            for i in range(n):
                cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
            #print(cf_mat)


            train_cr.append(correct_num/n)


            ### test set correct-ratio ###
            correct_num = 0
            esti_mat_test = np.zeros(n_test)
            for i in range(n_test):
                esti = np.dot(my_x_test[i,:], my_omega) + my_b
                esti_mat_test[i] = esti.argmax()    ###
                if esti.argmax() == my_y_test[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat_test = np.zeros((k,k))
            for i in range(n_test):
                cf_mat_test[int(my_y_test[i]),int(esti_mat_test[i])] += 1
            #print(cf_mat)

            test_cr.append(correct_num/n_test)
            test_f1.append(f1_score(my_y_test, esti_mat_test, average='macro')) ###
            
    
        except BaseException:
            print('can not calculate')
            if(len(test_cr) == 0):
                break
            print(train_cr[-1])
            print(cf_mat)
            print(max(test_cr))
            print(cf_mat_test)
            
            break
        
    if((iter == (iter_num - 1)) and (iter != 0)):
        print(train_cr[-1])
        print(cf_mat)

        print(max(test_cr))
        print(cf_mat_test)
        r_s5_test.append(max(test_cr))
        f_s5_test.append(max(test_f1))
        

1
0.44666666666666666
[[24. 17.  5. 11. 12.]
 [ 3.  5. 24.  7.  2.]
 [ 5.  1. 41.  9. 17.]
 [11. 10. 11. 31.  3.]
 [ 6.  6.  6.  0. 33.]]
0.48
[[ 9.  2.  1.  3.  5.]
 [ 3.  1. 12.  2.  1.]
 [ 0.  0. 14.  0.  7.]
 [ 4.  5.  3.  8.  3.]
 [ 2.  1.  1.  1. 12.]]
2
0.44
[[20. 13.  0.  4. 14.]
 [ 8.  6. 31. 16.  3.]
 [11.  2. 46.  6.  8.]
 [ 1. 10.  6. 29. 16.]
 [ 8.  9.  2.  0. 31.]]
0.49
[[ 3.  5.  0.  1.  8.]
 [ 3.  0.  8.  4.  1.]
 [ 5.  0. 22.  1.  4.]
 [ 1.  1.  2.  9.  3.]
 [ 2.  2.  0.  0. 15.]]
3
0.3433333333333333
[[16. 15.  3. 16. 22.]
 [18.  2.  9.  6. 11.]
 [27.  4. 35.  2. 14.]
 [ 4.  8.  7. 30.  3.]
 [ 6. 10. 11.  1. 20.]]
0.51
[[ 1.  7.  0.  5.  7.]
 [ 9.  0.  4.  1.  2.]
 [ 9.  1. 15.  0.  9.]
 [ 0.  2.  2.  7.  2.]
 [ 0.  7.  1.  0.  9.]]
4
can not calculate
0.16
[[21. 10.  8.  8.  7.]
 [20.  4.  9.  7. 11.]
 [29. 16. 12. 14. 19.]
 [16. 16. 10.  9. 10.]
 [14.  6. 11. 11.  2.]]
0.18
[[4. 8. 1. 2. 2.]
 [7. 5. 2. 4. 3.]
 [6. 3. 1. 6. 9.]
 [7. 3. 3. 7. 4.]
 [4. 0. 4. 4. 1.]]
5


In [30]:
print(r_s5_test)
print(np.mean(r_s5_test))
print(np.std(r_s5_test))

print(f_s5_test)
print(np.mean(f_s5_test))
print(np.std(f_s5_test))

[0.3, 0.3, 0.33, 0.32, 0.3, 0.46, 0.49, 0.51, 0.54, 0.48, 0.49, 0.51, 0.58]
0.4315384615384615
0.10029542161850229
[0.2332791018505304, 0.2759182567525171, 0.29674063525127353, 0.30464316851413625, 0.276715593500722, 0.43368504613091624, 0.44290885585003237, 0.461025641025641, 0.4898144256431278, 0.45058438951654534, 0.4470035259225316, 0.461025641025641, 0.5218772022551093]
0.39194011409528645
0.09422355003900332


## scalar = 10

In [24]:
for label in range(rep):
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s10_soap/'
    file_data = rountine + 'all_data_0309(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])

    file_data = rountine + 'y_label_0309(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0309(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n_test = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0309(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y_test),1))
    
    
    
    
    my_omega = np.random.randn(x_dim,k)
    my_b = np.random.randn(k)
    C_num = 0.2
    correct_num = 0
    iter_num = 15  ###总迭代次数

    train_cr = []
    test_cr = []
    test_f1 = []
    
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        if esti.argmax() == my_y[i]:
            correct_num += 1

    for iter in range(iter_num):
        try:
            my_beta = np.zeros((n,k))
            ##### beta
            for i in range(n):
                pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
                pred_copy = pred_iter.copy()
                pred_iter[int(my_y[i])] = -float('inf')
                j = pred_iter.argmax()

                if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
                    my_beta[i,j] = C_num

            ##### function1
            my_alpha = cp.Variable((n,k))
            obj_function = 0

            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

                s1 += cp.sum_squares(s1_1 - s1_2)

            s2 = 0
            for i in range(n):
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s2 += my_alpha[i,j2]

            obj_function = 0.5 * s1 - s2

            ##### constrains1
            constrains = []
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j])

                constrains += [(s1_1 - s1_2) == 0]

            for i in range(n):
                s1 = 0
                for j in range(k):
                    if j != my_y[i]:
                        s1 += my_alpha[i,j]
                        constrains += [s1 >= 0]
                        constrains += [s1 <= C_num]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains += [my_alpha[i,j] >= 0]

            prob = cp.Problem(cp.Minimize(obj_function), constrains)
            prob.solve()

            ##### calculate omega
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                my_omega[:,j] = s1_1 - s1_2

            #print(my_omega)

            #####function2
            my_eta = cp.Variable(n)
            my_b = cp.Variable(k)

            obj_function_2 = 0
            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += my_beta[i,j2]

                    if my_y[i] != j:
                        s1_2 += my_beta[i,j2]

                s1 += (s1_1 - s1_2) * my_b[j]

            obj_function_2 = C_num * cp.sum(my_eta) + s1

            ##### constrains2

            constrains_2 = []
            f_sim = np.dot(my_x, my_omega)

            for i in range(n):
                constrains_2 += [my_eta[i] >= 0]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

            constrains_2 += [cp.sum(my_b) == 0]

            prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)

            prob_2.solve()
            my_b = my_b.value
            #print(my_b)

            ### train set correct-ratio ###

            correct_num = 0
            esti_mat = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x[i,:], my_omega) + my_b
                esti_mat[i] = esti.argmax()
                if esti.argmax() == my_y[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat = np.zeros((k,k))
            for i in range(n):
                cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
            #print(cf_mat)


            train_cr.append(correct_num/n)


            ### test set correct-ratio ###
            correct_num = 0
            esti_mat_test = np.zeros(n_test)
            for i in range(n_test):
                esti = np.dot(my_x_test[i,:], my_omega) + my_b
                esti_mat_test[i] = esti.argmax()
                if esti.argmax() == my_y_test[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat_test = np.zeros((k,k))
            for i in range(n_test):
                cf_mat_test[int(my_y_test[i]),int(esti_mat_test[i])] += 1
            #print(cf_mat_test)

            test_cr.append(correct_num/n_test)
            test_f1.append(f1_score(my_y_test, esti_mat_test, average='macro')) ###
    
        except BaseException:
            print('can not calculate')
            if(len(test_cr) == 0):
                break
            print(train_cr)
            print(cf_mat)
            print(test_cr)
            print(cf_mat_test)
            
            break
        
    if((iter == (iter_num - 1)) and (iter != 0)):
        print(train_cr)
        print(cf_mat)

        print(test_cr)
        print(cf_mat_test)
        r_s10_test.append(max(test_cr))
        f_s10_test.append(max(test_f1))

1
[0.27666666666666667, 0.45, 0.32, 0.49666666666666665, 0.2733333333333333, 0.46, 0.3, 0.4666666666666667, 0.37333333333333335, 0.5066666666666667, 0.33666666666666667, 0.47, 0.32666666666666666, 0.45666666666666667, 0.4033333333333333]
[[13.  8.  1. 21. 20.]
 [15.  2. 21.  7.  3.]
 [26.  3. 36.  4. 13.]
 [ 7.  7.  5. 34.  2.]
 [ 2. 10.  4.  0. 36.]]
[0.29, 0.39, 0.24, 0.4, 0.2, 0.36, 0.21, 0.35, 0.26, 0.42, 0.29, 0.39, 0.22, 0.32, 0.33]
[[ 4.  7.  1.  4.  4.]
 [11.  1.  8.  4.  0.]
 [ 4.  1. 10.  0.  4.]
 [ 2.  3.  2. 10.  3.]
 [ 4.  4.  1.  0.  8.]]
2
[0.21666666666666667, 0.33666666666666667, 0.41333333333333333, 0.4166666666666667, 0.38333333333333336, 0.39666666666666667, 0.35333333333333333, 0.42, 0.38666666666666666, 0.41333333333333333, 0.42, 0.39, 0.3933333333333333, 0.37, 0.34]
[[16.  6.  0. 18. 15.]
 [28.  2. 13.  7.  2.]
 [22.  6. 39.  0. 15.]
 [ 3.  9.  9. 17. 15.]
 [ 5. 15.  5.  5. 28.]]
[0.15, 0.35, 0.37, 0.39, 0.39, 0.46, 0.28, 0.41, 0.39, 0.45, 0.44, 0.41, 0.43, 0.33,

D:\anaconda3\envs\env-cvxpy\lib\site-packages\cvxpy\problems\problem.py:1334: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


can not calculate
[0.27, 0.25333333333333335]
[[21.  7. 12. 12. 13.]
 [15.  6. 11.  8.  7.]
 [11. 15. 22. 16. 20.]
 [ 7.  8.  7. 19. 10.]
 [20. 10. 11.  4.  8.]]
[0.21, 0.17]
[[6. 7. 4. 3. 3.]
 [5. 2. 3. 5. 0.]
 [6. 6. 6. 5. 3.]
 [2. 6. 8. 1. 5.]
 [3. 3. 5. 1. 2.]]
4
[0.17333333333333334, 0.4866666666666667, 0.4633333333333333, 0.49, 0.5166666666666667, 0.34, 0.43666666666666665, 0.49333333333333335, 0.4, 0.4666666666666667, 0.4, 0.46, 0.48, 0.44333333333333336, 0.47333333333333333]
[[26. 19.  0.  9. 16.]
 [ 3.  3. 30.  8.  1.]
 [ 9.  4. 66. 10. 14.]
 [ 3.  2.  6. 26.  4.]
 [ 9.  8.  2.  1. 21.]]
[0.14, 0.53, 0.52, 0.57, 0.55, 0.32, 0.49, 0.52, 0.42, 0.47, 0.44, 0.5, 0.51, 0.46, 0.52]
[[ 5.  2.  0.  2.  5.]
 [ 2.  0. 10.  3.  0.]
 [ 4.  1. 27.  0.  5.]
 [ 0.  4.  2. 10.  2.]
 [ 3.  3.  0.  0. 10.]]
5
[0.35, 0.47, 0.26, 0.42333333333333334, 0.38333333333333336, 0.42, 0.44333333333333336, 0.41333333333333333, 0.46, 0.27666666666666667, 0.4266666666666667, 0.36666666666666664, 0.43, 0.453

In [25]:
print(r_s10_test)
print(np.mean(r_s10_test))
print(np.std(r_s10_test))

print(f_s10_test)
print(np.mean(f_s10_test))
print(np.std(f_s10_test))

[0.3, 0.33, 0.34, 0.43, 0.31, 0.45, 0.54, 0.53, 0.53, 0.58, 0.51, 0.42, 0.46, 0.57, 0.48]
0.45199999999999996
0.09188398482144026
[0.2745914560156046, 0.3351338498047776, 0.2635345047109753, 0.3681788232463659, 0.26229694629694633, 0.4522144522144522, 0.5055022496882963, 0.5278844459675842, 0.5002599553998088, 0.5827229319912247, 0.49725204373091697, 0.41828394100119926, 0.4550570087189806, 0.5143184743184743, 0.4666606956262129]
0.4282594519154547
0.10018005559723407


In [26]:
x_dim

13

## s (-0.25)

In [7]:
k = 5
s = -1/(k-1)
C_num = 3
rep = 5

In [8]:
s

-0.25

In [9]:
r_s2_test = []
r_s5_test = []
r_s10_test = []
r_s10_l2_test = []

In [10]:
f_s2_test = []
f_s5_test = []
f_s10_test = []
f_s10_l2_test = []

### s2

In [11]:
for label in range(rep):
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s2_soap/'
    file_data = rountine + 'all_data_0309(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])

    file_data = rountine + 'y_label_0309(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0309(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n_test = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0309(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y_test),1))
    
    
    
    
    my_omega = np.random.randn(x_dim,k)
    my_b = np.random.randn(k)
    C_num = 0.2
    correct_num = 0
    iter_num = 15  ###总迭代次数

    train_cr = []
    test_cr = []
    test_f1 = [] ######
    
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        if esti.argmax() == my_y[i]:
            correct_num += 1

    for iter in range(iter_num):
        try:
            my_beta = np.zeros((n,k))
            ##### beta
            for i in range(n):
                pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
                pred_copy = pred_iter.copy()
                pred_iter[int(my_y[i])] = -float('inf')
                j = pred_iter.argmax()

                if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
                    my_beta[i,j] = C_num

            ##### function1
            my_alpha = cp.Variable((n,k))
            obj_function = 0

            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

                s1 += cp.sum_squares(s1_1 - s1_2)

            s2 = 0
            for i in range(n):
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s2 += my_alpha[i,j2]

            obj_function = 0.5 * s1 - s2

            ##### constrains1
            constrains = []
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j])

                constrains += [(s1_1 - s1_2) == 0]

            for i in range(n):
                s1 = 0
                for j in range(k):
                    if j != my_y[i]:
                        s1 += my_alpha[i,j]
                        constrains += [s1 >= 0]
                        constrains += [s1 <= C_num]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains += [my_alpha[i,j] >= 0]

            prob = cp.Problem(cp.Minimize(obj_function), constrains)
            prob.solve()

            ##### calculate omega
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                my_omega[:,j] = s1_1 - s1_2

            #print(my_omega)

            #####function2
            my_eta = cp.Variable(n)
            my_b = cp.Variable(k)

            obj_function_2 = 0
            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += my_beta[i,j2]

                    if my_y[i] != j:
                        s1_2 += my_beta[i,j2]

                s1 += (s1_1 - s1_2) * my_b[j]

            obj_function_2 = C_num * cp.sum(my_eta) + s1

            ##### constrains2

            constrains_2 = []
            f_sim = np.dot(my_x, my_omega)

            for i in range(n):
                constrains_2 += [my_eta[i] >= 0]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

            constrains_2 += [cp.sum(my_b) == 0]

            prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)

            prob_2.solve()
            my_b = my_b.value
            #print(my_b)

            ### train set correct-ratio ###

            correct_num = 0
            esti_mat = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x[i,:], my_omega) + my_b
                esti_mat[i] = esti.argmax()
                if esti.argmax() == my_y[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat = np.zeros((k,k))
            for i in range(n):
                cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
            #print(cf_mat)


            train_cr.append(correct_num/n)


            ### test set correct-ratio ###
            correct_num = 0
            esti_mat_test = np.zeros(n_test)
            for i in range(n_test):
                esti = np.dot(my_x_test[i,:], my_omega) + my_b
                esti_mat_test[i] = esti.argmax()  ######
                if esti.argmax() == my_y_test[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat_test = np.zeros((k,k))
            for i in range(n_test):
                cf_mat_test[int(my_y_test[i]),int(esti_mat_test[i])] += 1
            #print(cf_mat)

            test_cr.append(correct_num/n_test)
            test_f1.append(f1_score(my_y_test, esti_mat_test, average='macro')) ######

    
        except BaseException:
            print('can not calculate')
            if(len(test_cr) == 0):
                break
            print(train_cr[-1])
            print(cf_mat)
            print(max(test_cr))
            print(cf_mat_test)
            
            break
        
    if((iter == (iter_num - 1)) and (iter != 0)):
        print(train_cr[-1])
        print(cf_mat)

        print(max(test_cr))
        print(cf_mat_test)
        
        r_s2_test.append(max(test_cr))  
        f_s2_test.append(max(test_f1))  ######

1
0.48333333333333334
[[ 8.  0.  0.  5. 16.]
 [ 0. 37.  1. 20.  4.]
 [ 0. 24. 44.  7.  6.]
 [15. 12.  1.  9. 15.]
 [ 7.  1. 16.  5. 47.]]
0.61
[[ 2.  0.  0.  1.  9.]
 [ 0.  8.  0.  8.  1.]
 [ 0. 10. 10.  5.  3.]
 [ 5.  3.  0.  7.  6.]
 [ 3.  0.  4.  3. 12.]]
2
0.55
[[ 2.  0.  2.  9. 26.]
 [ 8. 37.  8.  4.  0.]
 [13. 17. 56.  0.  0.]
 [ 9. 12.  3. 31.  6.]
 [ 3.  1. 12.  2. 39.]]
0.69
[[ 0.  0.  1.  4. 10.]
 [ 0.  7.  2.  0.  0.]
 [ 4.  6. 24.  0.  0.]
 [ 2.  2.  3. 13.  3.]
 [ 0.  0.  3.  0. 16.]]
3
0.55
[[ 7.  7.  1. 10. 26.]
 [ 8. 31.  2.  9.  0.]
 [ 6. 13. 53.  6.  0.]
 [ 6. 20.  0. 28.  2.]
 [ 2.  0. 15.  2. 46.]]
0.71
[[ 1.  1.  1.  2.  3.]
 [ 2. 10.  0.  3.  0.]
 [ 1.  5. 21.  3.  0.]
 [ 2.  5.  0. 14.  3.]
 [ 1.  0.  4.  0. 18.]]
4
0.48333333333333334
[[ 3. 13.  1.  8. 16.]
 [15.  1. 18. 18.  1.]
 [ 9.  3. 64.  2.  7.]
 [ 8.  8.  3. 27. 10.]
 [ 2. 11.  2.  0. 50.]]
0.6
[[ 1.  3.  0.  3.  2.]
 [ 3.  0.  4.  7.  0.]
 [ 3.  0. 25.  0.  5.]
 [ 5.  1.  0. 14.  4.]
 [ 0.  6.  0.  0. 1

In [12]:
print(r_s2_test)
print(np.mean(r_s2_test))
print(np.std(r_s2_test))

print(f_s2_test)
print(np.mean(f_s2_test))
print(np.std(f_s2_test))

[0.61, 0.69, 0.71, 0.6]
0.6525
0.04815340071064554
[0.5896727855496191, 0.6256629503141131, 0.6454602415128731, 0.5377635631366975]
0.5996398851283258
0.0409411995369401


### s5

In [15]:
for label in range(rep):
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s5_soap/'
    file_data = rountine + 'all_data_0309(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])

    file_data = rountine + 'y_label_0309(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0309(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n_test = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0309(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y_test),1))
    
    
    
    
    my_omega = np.random.randn(x_dim,k)
    my_b = np.random.randn(k)
    C_num = 0.2
    correct_num = 0
    iter_num = 15  ###总迭代次数

    train_cr = []
    test_cr = []
    test_f1 = [] ###
    
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        if esti.argmax() == my_y[i]:
            correct_num += 1

    for iter in range(iter_num):
        try:
            my_beta = np.zeros((n,k))
            ##### beta
            for i in range(n):
                pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
                pred_copy = pred_iter.copy()
                pred_iter[int(my_y[i])] = -float('inf')
                j = pred_iter.argmax()

                if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
                    my_beta[i,j] = C_num

            ##### function1
            my_alpha = cp.Variable((n,k))
            obj_function = 0

            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

                s1 += cp.sum_squares(s1_1 - s1_2)

            s2 = 0
            for i in range(n):
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s2 += my_alpha[i,j2]

            obj_function = 0.5 * s1 - s2

            ##### constrains1
            constrains = []
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j])

                constrains += [(s1_1 - s1_2) == 0]

            for i in range(n):
                s1 = 0
                for j in range(k):
                    if j != my_y[i]:
                        s1 += my_alpha[i,j]
                        constrains += [s1 >= 0]
                        constrains += [s1 <= C_num]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains += [my_alpha[i,j] >= 0]

            prob = cp.Problem(cp.Minimize(obj_function), constrains)
            prob.solve()

            ##### calculate omega
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                my_omega[:,j] = s1_1 - s1_2

            #print(my_omega)

            #####function2
            my_eta = cp.Variable(n)
            my_b = cp.Variable(k)

            obj_function_2 = 0
            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += my_beta[i,j2]

                    if my_y[i] != j:
                        s1_2 += my_beta[i,j2]

                s1 += (s1_1 - s1_2) * my_b[j]

            obj_function_2 = C_num * cp.sum(my_eta) + s1

            ##### constrains2

            constrains_2 = []
            f_sim = np.dot(my_x, my_omega)

            for i in range(n):
                constrains_2 += [my_eta[i] >= 0]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

            constrains_2 += [cp.sum(my_b) == 0]

            prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)

            prob_2.solve()
            my_b = my_b.value
            #print(my_b)

            ### train set correct-ratio ###

            correct_num = 0
            esti_mat = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x[i,:], my_omega) + my_b
                esti_mat[i] = esti.argmax()
                if esti.argmax() == my_y[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat = np.zeros((k,k))
            for i in range(n):
                cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
            #print(cf_mat)


            train_cr.append(correct_num/n)


            ### test set correct-ratio ###
            correct_num = 0
            esti_mat_test = np.zeros(n_test)
            for i in range(n_test):
                esti = np.dot(my_x_test[i,:], my_omega) + my_b
                esti_mat_test[i] = esti.argmax()    ###
                if esti.argmax() == my_y_test[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat_test = np.zeros((k,k))
            for i in range(n_test):
                cf_mat_test[int(my_y_test[i]),int(esti_mat_test[i])] += 1
            #print(cf_mat)

            test_cr.append(correct_num/n_test)
            test_f1.append(f1_score(my_y_test, esti_mat_test, average='macro')) ###
            
    
        except BaseException:
            print('can not calculate')
            if(len(test_cr) == 0):
                break
            print(train_cr[-1])
            print(cf_mat)
            print(max(test_cr))
            print(cf_mat_test)
            
            break
        
    if((iter == (iter_num - 1)) and (iter != 0)):
        print(train_cr[-1])
        print(cf_mat)

        print(max(test_cr))
        print(cf_mat_test)
        r_s5_test.append(max(test_cr))
        f_s5_test.append(max(test_f1))
        

1
0.43666666666666665
[[18. 18.  2. 17. 14.]
 [ 9.  4. 22.  2.  4.]
 [ 4.  5. 46.  6. 12.]
 [16.  8.  7. 30.  5.]
 [ 5. 10.  3.  0. 33.]]
0.43
[[ 4.  2.  1.  7.  6.]
 [ 1.  1. 15.  0.  2.]
 [ 1.  0. 14.  1.  5.]
 [ 3.  4.  4.  8.  4.]
 [ 1.  2.  1.  1. 12.]]
2
0.35333333333333333
[[10. 15.  1.  2. 23.]
 [ 3.  7. 29. 22.  3.]
 [ 1.  2. 40. 16. 14.]
 [ 5. 18.  2. 26. 11.]
 [15.  8.  4.  0. 23.]]
0.5
[[ 6.  6.  0.  0.  5.]
 [ 1.  3.  5.  5.  2.]
 [ 3.  0. 17.  6.  6.]
 [ 1.  3.  1.  6.  5.]
 [ 4.  2.  0.  0. 13.]]
3
0.43666666666666665
[[25. 14.  0. 18. 15.]
 [12.  2. 19. 13.  0.]
 [ 5.  8. 39.  8. 22.]
 [ 2. 10.  5. 34.  1.]
 [ 4.  9.  2.  2. 31.]]
0.47
[[ 4.  7.  0.  8.  1.]
 [ 5.  0.  7.  4.  0.]
 [ 0.  2. 18.  5.  9.]
 [ 0.  2.  2.  7.  2.]
 [ 0.  7.  1.  1.  8.]]
4
can not calculate
0.22
[[11. 16.  9. 12.  6.]
 [ 6. 13. 14.  8. 10.]
 [13. 18. 28. 16. 15.]
 [14. 12. 16. 10.  9.]
 [ 5. 14.  9. 12.  4.]]
0.34
[[ 7.  1.  4.  1.  4.]
 [ 3.  4.  7.  1.  6.]
 [ 2.  3. 16.  0.  4.]
 [ 2.  6.

In [16]:
print(r_s5_test)
print(np.mean(r_s5_test))
print(np.std(r_s5_test))

print(f_s5_test)
print(np.mean(f_s5_test))
print(np.std(f_s5_test))

[0.5, 0.48, 0.51, 0.58, 0.43, 0.5, 0.47, 0.55]
0.5025
0.04351723796382303
[0.46933827204045037, 0.44005025707153367, 0.4464264603970487, 0.5674162998532747, 0.4001182143219446, 0.4222698684962835, 0.41528917758152684, 0.5160720597716842]
0.4596225761917183
0.05286667551130658


## scalar = 10

In [17]:
for label in range(rep):
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s10_soap/'
    file_data = rountine + 'all_data_0309(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])

    file_data = rountine + 'y_label_0309(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0309(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n_test = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0309(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y_test),1))
    
    
    
    
    my_omega = np.random.randn(x_dim,k)
    my_b = np.random.randn(k)
    C_num = 0.2
    correct_num = 0
    iter_num = 15  ###总迭代次数

    train_cr = []
    test_cr = []
    test_f1 = []
    
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        if esti.argmax() == my_y[i]:
            correct_num += 1

    for iter in range(iter_num):
        try:
            my_beta = np.zeros((n,k))
            ##### beta
            for i in range(n):
                pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
                pred_copy = pred_iter.copy()
                pred_iter[int(my_y[i])] = -float('inf')
                j = pred_iter.argmax()

                if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
                    my_beta[i,j] = C_num

            ##### function1
            my_alpha = cp.Variable((n,k))
            obj_function = 0

            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

                s1 += cp.sum_squares(s1_1 - s1_2)

            s2 = 0
            for i in range(n):
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s2 += my_alpha[i,j2]

            obj_function = 0.5 * s1 - s2

            ##### constrains1
            constrains = []
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j])

                constrains += [(s1_1 - s1_2) == 0]

            for i in range(n):
                s1 = 0
                for j in range(k):
                    if j != my_y[i]:
                        s1 += my_alpha[i,j]
                        constrains += [s1 >= 0]
                        constrains += [s1 <= C_num]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains += [my_alpha[i,j] >= 0]

            prob = cp.Problem(cp.Minimize(obj_function), constrains)
            prob.solve()

            ##### calculate omega
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                my_omega[:,j] = s1_1 - s1_2

            #print(my_omega)

            #####function2
            my_eta = cp.Variable(n)
            my_b = cp.Variable(k)

            obj_function_2 = 0
            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += my_beta[i,j2]

                    if my_y[i] != j:
                        s1_2 += my_beta[i,j2]

                s1 += (s1_1 - s1_2) * my_b[j]

            obj_function_2 = C_num * cp.sum(my_eta) + s1

            ##### constrains2

            constrains_2 = []
            f_sim = np.dot(my_x, my_omega)

            for i in range(n):
                constrains_2 += [my_eta[i] >= 0]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

            constrains_2 += [cp.sum(my_b) == 0]

            prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)

            prob_2.solve()
            my_b = my_b.value
            #print(my_b)

            ### train set correct-ratio ###

            correct_num = 0
            esti_mat = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x[i,:], my_omega) + my_b
                esti_mat[i] = esti.argmax()
                if esti.argmax() == my_y[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat = np.zeros((k,k))
            for i in range(n):
                cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
            #print(cf_mat)


            train_cr.append(correct_num/n)


            ### test set correct-ratio ###
            correct_num = 0
            esti_mat_test = np.zeros(n_test)
            for i in range(n_test):
                esti = np.dot(my_x_test[i,:], my_omega) + my_b
                esti_mat_test[i] = esti.argmax()
                if esti.argmax() == my_y_test[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat_test = np.zeros((k,k))
            for i in range(n_test):
                cf_mat_test[int(my_y_test[i]),int(esti_mat_test[i])] += 1
            #print(cf_mat_test)

            test_cr.append(correct_num/n_test)
            test_f1.append(f1_score(my_y_test, esti_mat_test, average='macro')) ###
    
        except BaseException:
            print('can not calculate')
            if(len(test_cr) == 0):
                break
            print(train_cr)
            print(cf_mat)
            print(test_cr)
            print(cf_mat_test)
            
            break
        
    if((iter == (iter_num - 1)) and (iter != 0)):
        print(train_cr)
        print(cf_mat)

        print(test_cr)
        print(cf_mat_test)
        r_s10_test.append(max(test_cr))
        f_s10_test.append(max(test_f1))

1
[0.38333333333333336, 0.4666666666666667, 0.5866666666666667, 0.56, 0.5666666666666667, 0.5466666666666666, 0.43333333333333335, 0.45, 0.4533333333333333, 0.2866666666666667, 0.47, 0.4266666666666667, 0.43333333333333335, 0.3433333333333333, 0.4]
[[20. 14.  1. 16. 12.]
 [10.  3. 14. 15.  6.]
 [16.  5. 35.  3. 23.]
 [ 2. 11.  7. 32.  3.]
 [ 8. 12.  2.  0. 30.]]
[0.29, 0.47, 0.58, 0.54, 0.56, 0.51, 0.36, 0.45, 0.38, 0.2, 0.41, 0.31, 0.31, 0.26, 0.29]
[[ 4. 10.  1.  3.  2.]
 [ 4.  1.  8.  6.  5.]
 [ 5.  1.  9.  0.  4.]
 [ 3.  4.  1.  9.  3.]
 [ 3.  5.  3.  0.  6.]]
2
[0.4033333333333333, 0.33, 0.41333333333333333, 0.4033333333333333, 0.4, 0.4533333333333333, 0.4533333333333333, 0.37333333333333335, 0.43, 0.41, 0.30333333333333334, 0.42333333333333334, 0.39, 0.36333333333333334, 0.4033333333333333]
[[22.  9.  1.  6. 17.]
 [14.  3. 16. 18.  1.]
 [12.  5. 36. 18. 11.]
 [ 1. 14.  2. 31.  5.]
 [ 7. 16.  6.  0. 29.]]
[0.36, 0.24, 0.42, 0.42, 0.43, 0.41, 0.49, 0.26, 0.39, 0.44, 0.23, 0.41, 0.4

D:\anaconda3\envs\env-cvxpy\lib\site-packages\cvxpy\problems\problem.py:1334: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
D:\anaconda3\envs\env-cvxpy\lib\site-packages\cvxpy\problems\problem.py:1334: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "
D:\anaconda3\envs\env-cvxpy\lib\site-packages\cvxpy\problems\problem.py:1334: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


can not calculate
[0.18, 0.30666666666666664, 0.31333333333333335, 0.24333333333333335]
[[12. 11. 16. 13. 13.]
 [ 7. 12. 10. 12.  6.]
 [10. 20. 13. 16. 25.]
 [16.  9.  1. 15. 10.]
 [ 4.  3. 16.  9. 21.]]
[0.19, 0.32, 0.16, 0.24]
[[8. 2. 5. 3. 5.]
 [4. 2. 2. 7. 0.]
 [7. 5. 7. 3. 4.]
 [2. 9. 4. 1. 6.]
 [1. 2. 4. 1. 6.]]
4
[0.2833333333333333, 0.38, 0.4766666666666667, 0.48333333333333334, 0.38666666666666666, 0.4533333333333333, 0.47333333333333333, 0.44, 0.43666666666666665, 0.4533333333333333, 0.51, 0.36, 0.45666666666666667, 0.39, 0.42333333333333334]
[[24. 21.  0. 15. 10.]
 [ 2.  3. 27.  7.  6.]
 [ 5.  7. 57. 11. 23.]
 [ 3.  4.  4. 22.  8.]
 [ 8. 10.  2.  0. 21.]]
[0.32, 0.43, 0.47, 0.47, 0.4, 0.48, 0.51, 0.46, 0.43, 0.48, 0.45, 0.32, 0.49, 0.39, 0.45]
[[ 5.  4.  0.  1.  4.]
 [ 0.  2.  8.  5.  0.]
 [ 2.  1. 24.  2.  8.]
 [ 0.  6.  1. 10.  1.]
 [ 9.  3.  0.  0.  4.]]
5
[0.29, 0.24333333333333335, 0.3333333333333333, 0.44, 0.38333333333333336, 0.29333333333333333, 0.39, 0.3766666666666

In [18]:
print(r_s10_test)
print(np.mean(r_s10_test))
print(np.std(r_s10_test))

print(f_s10_test)
print(np.mean(f_s10_test))
print(np.std(f_s10_test))

[0.58, 0.49, 0.51, 0.49]
0.5175
0.03699662146737184
[0.5692602495543672, 0.45140113509678725, 0.4556248137010387, 0.4665252676805565]
0.48570286650818745
0.048556448070284704


In [19]:
x_dim

13

# s=0.025

In [20]:
k = 5
s = -1/(k-1)/10
C_num = 3
rep = 5

In [21]:
s

-0.025

In [22]:
r_s2_test = []
r_s5_test = []
r_s10_test = []
r_s10_l2_test = []

In [23]:
f_s2_test = []
f_s5_test = []
f_s10_test = []
f_s10_l2_test = []

### s2

In [24]:
for label in range(rep):
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s2_soap/'
    file_data = rountine + 'all_data_0309(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])

    file_data = rountine + 'y_label_0309(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0309(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n_test = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0309(2)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y_test),1))
    
    
    
    
    my_omega = np.random.randn(x_dim,k)
    my_b = np.random.randn(k)
    C_num = 0.2
    correct_num = 0
    iter_num = 15  ###总迭代次数

    train_cr = []
    test_cr = []
    test_f1 = [] ######
    
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        if esti.argmax() == my_y[i]:
            correct_num += 1

    for iter in range(iter_num):
        try:
            my_beta = np.zeros((n,k))
            ##### beta
            for i in range(n):
                pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
                pred_copy = pred_iter.copy()
                pred_iter[int(my_y[i])] = -float('inf')
                j = pred_iter.argmax()

                if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
                    my_beta[i,j] = C_num

            ##### function1
            my_alpha = cp.Variable((n,k))
            obj_function = 0

            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

                s1 += cp.sum_squares(s1_1 - s1_2)

            s2 = 0
            for i in range(n):
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s2 += my_alpha[i,j2]

            obj_function = 0.5 * s1 - s2

            ##### constrains1
            constrains = []
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j])

                constrains += [(s1_1 - s1_2) == 0]

            for i in range(n):
                s1 = 0
                for j in range(k):
                    if j != my_y[i]:
                        s1 += my_alpha[i,j]
                        constrains += [s1 >= 0]
                        constrains += [s1 <= C_num]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains += [my_alpha[i,j] >= 0]

            prob = cp.Problem(cp.Minimize(obj_function), constrains)
            prob.solve()

            ##### calculate omega
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                my_omega[:,j] = s1_1 - s1_2

            #print(my_omega)

            #####function2
            my_eta = cp.Variable(n)
            my_b = cp.Variable(k)

            obj_function_2 = 0
            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += my_beta[i,j2]

                    if my_y[i] != j:
                        s1_2 += my_beta[i,j2]

                s1 += (s1_1 - s1_2) * my_b[j]

            obj_function_2 = C_num * cp.sum(my_eta) + s1

            ##### constrains2

            constrains_2 = []
            f_sim = np.dot(my_x, my_omega)

            for i in range(n):
                constrains_2 += [my_eta[i] >= 0]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

            constrains_2 += [cp.sum(my_b) == 0]

            prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)

            prob_2.solve()
            my_b = my_b.value
            #print(my_b)

            ### train set correct-ratio ###

            correct_num = 0
            esti_mat = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x[i,:], my_omega) + my_b
                esti_mat[i] = esti.argmax()
                if esti.argmax() == my_y[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat = np.zeros((k,k))
            for i in range(n):
                cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
            #print(cf_mat)


            train_cr.append(correct_num/n)


            ### test set correct-ratio ###
            correct_num = 0
            esti_mat_test = np.zeros(n_test)
            for i in range(n_test):
                esti = np.dot(my_x_test[i,:], my_omega) + my_b
                esti_mat_test[i] = esti.argmax()  ######
                if esti.argmax() == my_y_test[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat_test = np.zeros((k,k))
            for i in range(n_test):
                cf_mat_test[int(my_y_test[i]),int(esti_mat_test[i])] += 1
            #print(cf_mat)

            test_cr.append(correct_num/n_test)
            test_f1.append(f1_score(my_y_test, esti_mat_test, average='macro')) ######

    
        except BaseException:
            print('can not calculate')
            if(len(test_cr) == 0):
                break
            print(train_cr[-1])
            print(cf_mat)
            print(max(test_cr))
            print(cf_mat_test)
            
            break
        
    if((iter == (iter_num - 1)) and (iter != 0)):
        print(train_cr[-1])
        print(cf_mat)

        print(max(test_cr))
        print(cf_mat_test)
        
        r_s2_test.append(max(test_cr))  
        f_s2_test.append(max(test_f1))  ######

1
0.58
[[ 3.  0.  0. 17.  9.]
 [15. 37.  9.  1.  0.]
 [ 4. 15. 42.  0. 20.]
 [11. 14.  1. 26.  0.]
 [ 5.  0.  3.  2. 66.]]
0.59
[[ 1.  0.  0.  9.  2.]
 [ 5.  8.  4.  0.  0.]
 [ 2.  4. 17.  0.  5.]
 [ 6.  5.  0. 10.  0.]
 [ 4.  0.  0.  2. 16.]]
2
0.53
[[ 6.  4.  2.  9. 18.]
 [ 0. 25. 18. 14.  0.]
 [ 0.  9. 65.  1. 11.]
 [ 3. 31.  3. 21.  3.]
 [ 3.  1.  9.  2. 42.]]
0.72
[[ 1.  5.  0.  3.  6.]
 [ 0.  4.  4.  1.  0.]
 [ 0.  3. 26.  0.  5.]
 [ 0. 12.  1. 10.  0.]
 [ 0.  1.  0.  0. 18.]]
3
0.54
[[13.  1.  3. 19. 15.]
 [ 1. 32.  6. 10.  1.]
 [ 0. 10. 51.  5. 12.]
 [ 7. 24.  2. 22.  1.]
 [ 1.  0. 17.  3. 44.]]
0.71
[[ 2.  0.  0.  2.  4.]
 [ 0. 11.  2.  2.  0.]
 [ 0.  5. 24.  0.  1.]
 [ 3.  6.  1. 14.  0.]
 [ 0.  0.  6.  0. 17.]]
4
0.49666666666666665
[[12. 14.  0.  3. 12.]
 [ 0.  1. 21. 27.  4.]
 [ 0.  3. 69.  4.  9.]
 [11.  9.  3. 19. 14.]
 [ 2.  8.  7.  0. 48.]]
0.56
[[ 4.  3.  0.  1.  1.]
 [ 0.  0.  6.  8.  0.]
 [ 0.  0. 28.  0.  5.]
 [ 5.  3.  5. 10.  1.]
 [ 0.  5.  1.  0. 14.]]
5


D:\anaconda3\envs\env-cvxpy\lib\site-packages\cvxpy\problems\problem.py:1334: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  "Solution may be inaccurate. Try another solver, "


can not calculate
0.21333333333333335
[[ 8.  6.  7. 12.  5.]
 [14. 11. 10. 10.  9.]
 [17. 12. 27. 14. 19.]
 [11.  8. 17. 11.  9.]
 [ 8. 19.  9. 20.  7.]]
0.22
[[3. 2. 0. 2. 1.]
 [3. 4. 6. 1. 6.]
 [9. 4. 3. 5. 4.]
 [3. 6. 7. 7. 8.]
 [1. 1. 6. 3. 5.]]


In [25]:
print(r_s2_test)
print(np.mean(r_s2_test))
print(np.std(r_s2_test))

print(f_s2_test)
print(np.mean(f_s2_test))
print(np.std(f_s2_test))

[0.59, 0.72, 0.71, 0.56]
0.645
0.0708872343937891
[0.5677518896054365, 0.6553865805456474, 0.6552380952380952, 0.47192216809074744]
0.5875746833699816
0.07573842840203746


### s5

In [26]:
for label in range(rep):
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s5_soap/'
    file_data = rountine + 'all_data_0309(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])

    file_data = rountine + 'y_label_0309(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0309(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n_test = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0309(5)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y_test),1))
    
    
    
    
    my_omega = np.random.randn(x_dim,k)
    my_b = np.random.randn(k)
    C_num = 0.2
    correct_num = 0
    iter_num = 15  ###总迭代次数

    train_cr = []
    test_cr = []
    test_f1 = [] ###
    
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        if esti.argmax() == my_y[i]:
            correct_num += 1

    for iter in range(iter_num):
        try:
            my_beta = np.zeros((n,k))
            ##### beta
            for i in range(n):
                pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
                pred_copy = pred_iter.copy()
                pred_iter[int(my_y[i])] = -float('inf')
                j = pred_iter.argmax()

                if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
                    my_beta[i,j] = C_num

            ##### function1
            my_alpha = cp.Variable((n,k))
            obj_function = 0

            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

                s1 += cp.sum_squares(s1_1 - s1_2)

            s2 = 0
            for i in range(n):
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s2 += my_alpha[i,j2]

            obj_function = 0.5 * s1 - s2

            ##### constrains1
            constrains = []
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j])

                constrains += [(s1_1 - s1_2) == 0]

            for i in range(n):
                s1 = 0
                for j in range(k):
                    if j != my_y[i]:
                        s1 += my_alpha[i,j]
                        constrains += [s1 >= 0]
                        constrains += [s1 <= C_num]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains += [my_alpha[i,j] >= 0]

            prob = cp.Problem(cp.Minimize(obj_function), constrains)
            prob.solve()

            ##### calculate omega
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                my_omega[:,j] = s1_1 - s1_2

            #print(my_omega)

            #####function2
            my_eta = cp.Variable(n)
            my_b = cp.Variable(k)

            obj_function_2 = 0
            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += my_beta[i,j2]

                    if my_y[i] != j:
                        s1_2 += my_beta[i,j2]

                s1 += (s1_1 - s1_2) * my_b[j]

            obj_function_2 = C_num * cp.sum(my_eta) + s1

            ##### constrains2

            constrains_2 = []
            f_sim = np.dot(my_x, my_omega)

            for i in range(n):
                constrains_2 += [my_eta[i] >= 0]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

            constrains_2 += [cp.sum(my_b) == 0]

            prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)

            prob_2.solve()
            my_b = my_b.value
            #print(my_b)

            ### train set correct-ratio ###

            correct_num = 0
            esti_mat = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x[i,:], my_omega) + my_b
                esti_mat[i] = esti.argmax()
                if esti.argmax() == my_y[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat = np.zeros((k,k))
            for i in range(n):
                cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
            #print(cf_mat)


            train_cr.append(correct_num/n)


            ### test set correct-ratio ###
            correct_num = 0
            esti_mat_test = np.zeros(n_test)
            for i in range(n_test):
                esti = np.dot(my_x_test[i,:], my_omega) + my_b
                esti_mat_test[i] = esti.argmax()    ###
                if esti.argmax() == my_y_test[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat_test = np.zeros((k,k))
            for i in range(n_test):
                cf_mat_test[int(my_y_test[i]),int(esti_mat_test[i])] += 1
            #print(cf_mat)

            test_cr.append(correct_num/n_test)
            test_f1.append(f1_score(my_y_test, esti_mat_test, average='macro')) ###
            
    
        except BaseException:
            print('can not calculate')
            if(len(test_cr) == 0):
                break
            print(train_cr[-1])
            print(cf_mat)
            print(max(test_cr))
            print(cf_mat_test)
            
            break
        
    if((iter == (iter_num - 1)) and (iter != 0)):
        print(train_cr[-1])
        print(cf_mat)

        print(max(test_cr))
        print(cf_mat_test)
        r_s5_test.append(max(test_cr))
        f_s5_test.append(max(test_f1))
        

1
0.5133333333333333
[[19. 11.  2. 12. 25.]
 [ 1. 28.  4.  6.  2.]
 [ 0. 14. 45. 10.  4.]
 [ 8.  0.  5. 35. 18.]
 [ 8.  1. 10.  5. 27.]]
0.54
[[ 8.  3.  0.  3.  6.]
 [ 0. 15.  3.  1.  0.]
 [ 0.  1. 15.  4.  1.]
 [ 4.  2.  1.  7.  9.]
 [ 2.  0.  4.  2.  9.]]
2
0.4066666666666667
[[ 8. 13.  0.  6. 24.]
 [21.  6. 14. 21.  2.]
 [15.  2. 41. 11.  4.]
 [ 3. 11.  3. 33. 12.]
 [ 2.  8.  6.  0. 34.]]
0.57
[[ 0.  6.  0.  2.  9.]
 [ 3.  0.  5.  7.  1.]
 [ 5.  0. 24.  3.  0.]
 [ 1.  1.  0. 12.  2.]
 [ 0.  2.  1.  1. 15.]]
3
0.44333333333333336
[[26. 16.  2. 18. 10.]
 [ 2.  2. 26. 14.  2.]
 [ 2.  7. 52.  8. 13.]
 [ 3.  9.  1. 25. 14.]
 [ 3.  6.  7.  4. 28.]]
0.45
[[ 6.  6.  2.  5.  1.]
 [ 1.  0.  9.  5.  1.]
 [ 1.  1. 25.  2.  5.]
 [ 0.  3.  1.  7.  2.]
 [ 1.  6.  1.  2.  7.]]
4
can not calculate
0.17666666666666667
[[ 9. 11.  6. 13. 15.]
 [ 6.  6. 12.  9. 18.]
 [19. 12. 11. 16. 32.]
 [ 9. 14.  4. 15. 19.]
 [ 8.  5.  4. 15. 12.]]
0.21
[[ 5.  7.  0.  1.  4.]
 [ 5.  3.  3.  3.  7.]
 [ 4.  2.  3.  2. 

In [27]:
print(r_s5_test)
print(np.mean(r_s5_test))
print(np.std(r_s5_test))

print(f_s5_test)
print(np.mean(f_s5_test))
print(np.std(f_s5_test))

[0.54, 0.57, 0.45, 0.58]
0.5349999999999999
0.051234753829797974
[0.5361955691367456, 0.5494199494199494, 0.38884893560349953, 0.5458211561437368]
0.5050714025759828
0.06727501690129155


## scalar = 10

In [28]:
for label in range(rep):
    label = label + 1
    print(label)
    rountine = 'D:/研究生文件/科研任务/functional_data/sim_data/s10_soap/'
    file_data = rountine + 'all_data_0309(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x = my_data[:,1:len(my_data[0,:])]
    
    n = len(my_x[:,0])
    x_dim = len(my_x[0,:])

    file_data = rountine + 'y_label_0309(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y = my_data[:,1] - 1 ##注意减一
    my_y = my_y.reshape((len(my_y),1))

    file_data = rountine + 'all_data_test_0309(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_x_test = my_data[:,1:len(my_data[0,:])]
    n_test = len(my_x_test[:,0])
    x_dim = len(my_x_test[0,:])

    file_data = rountine + 'y_label_test_0309(10)_' + str(label) + '.csv'
    data = pd.read_csv(file_data)
    my_data = data.values

    my_y_test = my_data[:,1] - 1 ##注意减一
    my_y_test = my_y_test.reshape((len(my_y_test),1))
    
    
    
    
    my_omega = np.random.randn(x_dim,k)
    my_b = np.random.randn(k)
    C_num = 0.2
    correct_num = 0
    iter_num = 15  ###总迭代次数

    train_cr = []
    test_cr = []
    test_f1 = []
    
    for i in range(n):
        esti = np.dot(my_x[i,:], my_omega) + my_b
        if esti.argmax() == my_y[i]:
            correct_num += 1

    for iter in range(iter_num):
        try:
            my_beta = np.zeros((n,k))
            ##### beta
            for i in range(n):
                pred_iter = (np.dot(my_x[i,:], my_omega) + my_b)
                pred_copy = pred_iter.copy()
                pred_iter[int(my_y[i])] = -float('inf')
                j = pred_iter.argmax()

                if((pred_copy[int(my_y[i])] - pred_copy[j]) < s):
                    my_beta[i,j] = C_num

            ##### function1
            my_alpha = cp.Variable((n,k))
            obj_function = 0

            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j]) *  my_x[i,:]

                s1 += cp.sum_squares(s1_1 - s1_2)

            s2 = 0
            for i in range(n):
                for j2 in range(k):
                    if j2 != my_y[i]:
                        s2 += my_alpha[i,j2]

            obj_function = 0.5 * s1 - s2

            ##### constrains1
            constrains = []
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha[i,j2] - my_beta[i,j2])

                    if my_y[i] != j:
                        s1_2 += (my_alpha[i,j] - my_beta[i,j])

                constrains += [(s1_1 - s1_2) == 0]

            for i in range(n):
                s1 = 0
                for j in range(k):
                    if j != my_y[i]:
                        s1 += my_alpha[i,j]
                        constrains += [s1 >= 0]
                        constrains += [s1 <= C_num]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains += [my_alpha[i,j] >= 0]

            prob = cp.Problem(cp.Minimize(obj_function), constrains)
            prob.solve()

            ##### calculate omega
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                    if my_y[i] != j:
                        s1_2 += (my_alpha.value[i,j2] - my_beta[i,j2]) *  my_x[i,:]

                my_omega[:,j] = s1_1 - s1_2

            #print(my_omega)

            #####function2
            my_eta = cp.Variable(n)
            my_b = cp.Variable(k)

            obj_function_2 = 0
            s1 = 0
            for j in range(k):
                s1_1 = 0
                s1_2 = 0
                for i in range(n):
                    if my_y[i] == j:
                        for j2 in range(k):
                            if j2 != my_y[i]:
                                s1_1 += my_beta[i,j2]

                    if my_y[i] != j:
                        s1_2 += my_beta[i,j2]

                s1 += (s1_1 - s1_2) * my_b[j]

            obj_function_2 = C_num * cp.sum(my_eta) + s1

            ##### constrains2

            constrains_2 = []
            f_sim = np.dot(my_x, my_omega)

            for i in range(n):
                constrains_2 += [my_eta[i] >= 0]

            for i in range(n):
                for j in range(k):
                    if j != my_y[i]:
                        constrains_2 += [my_eta[i] >= (1 - (f_sim[i,int(my_y[i])] + my_b[int(my_y[i])]) + f_sim[i,j] + my_b[j])]

            constrains_2 += [cp.sum(my_b) == 0]

            prob_2 = cp.Problem(cp.Minimize(obj_function_2), constrains_2)

            prob_2.solve()
            my_b = my_b.value
            #print(my_b)

            ### train set correct-ratio ###

            correct_num = 0
            esti_mat = np.zeros(n)
            for i in range(n):
                esti = np.dot(my_x[i,:], my_omega) + my_b
                esti_mat[i] = esti.argmax()
                if esti.argmax() == my_y[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat = np.zeros((k,k))
            for i in range(n):
                cf_mat[int(my_y[i]),int(esti_mat[i])] += 1
            #print(cf_mat)


            train_cr.append(correct_num/n)


            ### test set correct-ratio ###
            correct_num = 0
            esti_mat_test = np.zeros(n_test)
            for i in range(n_test):
                esti = np.dot(my_x_test[i,:], my_omega) + my_b
                esti_mat_test[i] = esti.argmax()
                if esti.argmax() == my_y_test[i]:
                    correct_num += 1
            #print(correct_num/n)

            cf_mat_test = np.zeros((k,k))
            for i in range(n_test):
                cf_mat_test[int(my_y_test[i]),int(esti_mat_test[i])] += 1
            #print(cf_mat_test)

            test_cr.append(correct_num/n_test)
            test_f1.append(f1_score(my_y_test, esti_mat_test, average='macro')) ###
    
        except BaseException:
            print('can not calculate')
            if(len(test_cr) == 0):
                break
            print(train_cr)
            print(cf_mat)
            print(test_cr)
            print(cf_mat_test)
            
            break
        
    if((iter == (iter_num - 1)) and (iter != 0)):
        print(train_cr)
        print(cf_mat)

        print(test_cr)
        print(cf_mat_test)
        r_s10_test.append(max(test_cr))
        f_s10_test.append(max(test_f1))

1
[0.37333333333333335, 0.30666666666666664, 0.42, 0.43333333333333335, 0.30333333333333334, 0.44333333333333336, 0.49333333333333335, 0.45666666666666667, 0.5233333333333333, 0.41, 0.5, 0.32666666666666666, 0.4166666666666667, 0.4633333333333333, 0.32666666666666666]
[[11. 12.  1. 16. 23.]
 [19.  3.  7. 15.  4.]
 [33.  3. 29.  4. 13.]
 [11. 11.  2. 28.  3.]
 [ 4. 11. 10.  0. 27.]]
[0.29, 0.26, 0.35, 0.34, 0.19, 0.33, 0.31, 0.34, 0.41, 0.28, 0.35, 0.21, 0.34, 0.33, 0.27]
[[ 4. 10.  0.  3.  3.]
 [11.  1.  5.  6.  1.]
 [ 6.  1.  6.  1.  5.]
 [ 3.  4.  2. 10.  1.]
 [ 5.  5.  1.  0.  6.]]
2
[0.33, 0.38666666666666666, 0.39666666666666667, 0.42333333333333334, 0.3566666666666667, 0.42333333333333334, 0.41, 0.4266666666666667, 0.41, 0.43333333333333335, 0.42, 0.38666666666666666, 0.37333333333333335, 0.43333333333333335, 0.4533333333333333]
[[30.  7.  1.  8.  9.]
 [13.  2. 19. 14.  4.]
 [ 5.  4. 39. 12. 22.]
 [ 4. 13.  1. 29.  6.]
 [ 1. 15.  6.  0. 36.]]
[0.32, 0.38, 0.45, 0.43, 0.33, 0.42, 

In [29]:
print(r_s10_test)
print(np.mean(r_s10_test))
print(np.std(r_s10_test))

print(f_s10_test)
print(np.mean(f_s10_test))
print(np.std(f_s10_test))

[0.41, 0.53, 0.49, 0.46]
0.4725
0.0438035386698381
[0.41325757575757577, 0.5070243795192495, 0.47132261600292297, 0.43915854667231535]
0.4576907794880159
0.03513326168211903
